# AWS CodeBuild
- AWS CodeBuild는 소스 코드를 컴파일하는 단계부터 테스트 실행 후 소프트웨어 패키지를 개발하여 배포하는 단계까지 마칠 수 있는 완전관리형의 지속적 통합 서비스입니다
- 빌드와 테스트를 서비스로 제공
    - 요청에 따라 컨테이너로 구성한 빌드/테스트 환경을 하나 생성
    - 이후 받은 소스를 기반으로 빌드/테스트를 수행해주는 서비스
- 환경 구성 커스터마이징 가능
    - 도커 이미지 / CPU, Memory / 환경 변수 / 제한 시간 / 파일 시스템(EFS)등 설정 가능
- AWS IAM 기반으로 다양한 AWS 서비스와 연동
    - 예 : CodeCommit / S3 등에서 소스 및 데이터 받아오기
    - 예 : CloudWatch / S3로 로그 관리
- 참고
    - 코드 빌드 람다 모드 : EC2 대신 람다로 빌드를 활용하는 모드

## AWS CodeBuild 설정
- AMI 이미지 : Amazon Linux, Windows, Ubuntu, macOS 등
- 컴퓨팅 인스턴스 유형 : 미리 설정된 인스턴스의 vCPU/Memory 등의 구성
    - 선택한 이미지에 따라 선택 가능한 인스턴스 유형이 다름
    - 프로비전 속도 차이 존재 : 수요가 다르기 때문
- VPC : VPC 설정 가능 -> 상황에 따라서 프라이빗 리소스에 접근이 필요한 경우

## CodeBuild 프로비전 모드
- 두 가지 프로비전 모드
    - 온디맨드 : 리소스가 필요할 때 준비해서 사용 후 종료
        - 프로비전 시간 필요
        - macOS와 MS Server 2022 사용 불가능
    - Reserved Capacity Fleet : 지정한 EC2를 지명하여 준비시키고 사용 후 내가 직접 종료
        - 처음 준비기간 이후 계속 환경이 준비되어 있어서 빌드가 빠름
        - macOS 사용 가능
        - 배치 빌드 불가능
        - 서울 리전 사용 불가능

### buildspec.yml
- 코드빌드에서 수행할 내용을 정의한 문서
- 반드시 소스의 루트 디렉토리에 위치해야 함
- 기본 이름 : buildspec.yml (Codebuild에서 변경 가능)
    - 예 : buildspec_debugyml / buildspec.prod.yml
- 정의 가능한 내용
    - 빌드 환경(node js, python), 환경 변수, 캐시
    - 스테이지 별 명령어
        - install, pre_build, build
    - 아티팩트 설정
        - path 및 구성 방법
    - 기타 (cache, proxy 등)

#### buildspec.yml의 phases
- install : 빌드/테스트 환경을 구성하기 위한 패키지의 다운로드 및 인스톨
    - node js, python을 설치한다던지..
- pre_build : 빌드/테스트 전 수행해야 하는 내용(디펜던시 다운로드/인스톨, 외부 리소스 확보 등)
- build : 실제 빌드/테스트 수행
- post_build : 빌드/테스트 이후 마무리 작업(도커 이미지 푸시, 슬랙 알람, 기록 작성 등)

## AWS CodeBuild Caching
- 두 가지 모드
    - Amazon S3 Caching : 서로 공유할 수 있는 버킷에 내용을 캐싱
        - 주로 작거나 미리 빌드해두면 좋은 패키지등을 저장
        - 네트워크를 사용하기 때문에 큰 파일의 경우 비효율적
    - Local Caching : 하나의 빌드 호스트에 로컬로 저장하는 캐시
        - 주로 크거나 바로 필요한 내용을 캐싱
        - 빌드가 빈번한 경우
        - 3가지 모드 : Source Cache / Docket Layer Cache / Custom Cache


### AWS CodeBuild Local Caching
- Source Cache
    - Primary / Secondary Source의 Git Metadata를 캐시
    - 캐시 생성 이후부터는 커밋의 변화 부분만 가져옴
    - 깃 소스코드 자체가 엄청 큰 상황일 때 (예: node_module이 깃에 있을 때)
- Docker Layer Cache
    - Docker Layer를 캐시
    - 큰 도커 이미지를 빌드하거나 가져올 때 (이미지를 가져오기 위한 네트워크 절약 가능)
    - 리눅스 환경만 가능
- Custom Cache Mode
    - buildspec에서 명시한 디렉토리만 캐시
    - 소스 다운로드 전에 세팅 -> 소스에 동일한 파일명이 있다면 덮어쓰기

### 코드빌드를 실전에서 사용할 때 생각할 점
- 프로비전에 시간이 오래 걸림
    - 실제 프로비전까지 10분까지 걸리는 경우도 발생
    - 환경 이미지/인스턴스 타입에 따라 프로비전 시간이 다름
- 비용
    - 꽤나 비용이 들어가는 서비스임